import pandas as pd

In [2]:
import pandas as pd

In [3]:
pantopoleio = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Pantopoleio Kali Orexi.csv', error_bad_lines=False);
syrianarab = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Syrian Arab Friendship Club.csv', error_bad_lines=False);

tripadv_review = pd.concat([pantopoleio, syrianarab])
tripadv_review

,r_points,r_text,place
0,40,"I recomend the place, is a restaurante with Go...",Pantopoleio Kali Orexi
1,40,Saturday lunch with family. Arrived early befo...,Pantopoleio Kali Orexi
2,50,"A very contemporary feel to this restaurant , ...",Pantopoleio Kali Orexi
3,40,"The restaurant is big, not fussy, plenty of sp...",Pantopoleio Kali Orexi
4,40,We have been many times to Pantopolio and stan...,Pantopoleio Kali Orexi
...,...,...,...
515,40,Tasty food. Good price.,Syrian Arab Friendship Club
516,10,The meze menu includes some dips and salads th...,Syrian Arab Friendship Club
517,50,"One of our favoutirte restaurants in Nicosia, ...",Syrian Arab Friendship Club
518,40,"its nice,clean,friendly.Not much variety.SImpl...",Syrian Arab Friendship Club


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [7]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0      [recomend, place, restaurante, good, service, ...
1      [saturday, lunch, family, arrived, early, crow...
2      [contemporary, feel, restaurant, decor, modern...
3      [restaurant, fussy, plenty, space, feel, comfo...
4      [times, pantopolio, standard, varies, expensiv...
                             ...                        
515                           [tasty, food, good, price]
516    [meze, menu, includes, dips, salads, look, lik...
517    [favoutirte, restaurants, nicosia, expect, bes...
518       [nice, clean, friendly, variety, simple, good]
519    [visited, damascus, tasted, syrian, food, syri...
Name: r_text, Length: 670, dtype: object

In [8]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=800,
                                           passes=8,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 64,
                                           eval_every = None)

In [10]:
from pprint import pprint

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.037*"food" + 0.037*"good" + 0.023*"service" + 0.013*"excellent" + '
  '0.012*"restaurant" + 0.012*"place" + 0.010*"atmosphere" + 0.009*"time" + '
  '0.008*"order" + 0.008*"nice"'),
 (1,
  '0.020*"food" + 0.017*"restaurant" + 0.010*"place" + 0.007*"order" + '
  '0.007*"service" + 0.007*"prices" + 0.007*"euro" + 0.006*"chicken" + '
  '0.006*"menu" + 0.006*"good"'),
 (2,
  '0.044*"food" + 0.038*"good" + 0.024*"nice" + 0.023*"service" + '
  '0.021*"place" + 0.019*"friendly" + 0.015*"staff" + 0.015*"prices" + '
  '0.012*"syrian" + 0.011*"atmosphere"'),
 (3,
  '0.019*"food" + 0.017*"good" + 0.017*"meat" + 0.011*"salads" + '
  '0.010*"nicosia" + 0.010*"excellent" + 0.010*"great" + 0.009*"restaurant" + '
  '0.008*"dips" + 0.008*"bread"'),
 (4,
  '0.034*"food" + 0.024*"syrian" + 0.019*"meze" + 0.014*"service" + '
  '0.014*"dishes" + 0.014*"great" + 0.014*"nice" + 0.011*"place" + '
  '0.010*"friendly" + 0.010*"atmosphere"'),
 (5,
  '0.036*"good" + 0.034*"food" + 0.026*"service" + 0.017

In [12]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.082339 -0.010363       1        1  20.607891
4      0.051526  0.023857       2        1  16.558702
6      0.045511 -0.023550       3        1  11.313462
0      0.034099  0.015810       4        1  10.212117
9      0.024397 -0.018980       5        1   9.829383
7      0.005376 -0.024639       6        1   9.293584
5      0.022472 -0.003432       7        1   9.171049
1     -0.056139  0.143167       8        1   6.629445
3     -0.051206 -0.067811       9        1   3.681933
8     -0.158375 -0.034060      10        1   2.702436, topic_info=           Term        Freq       Total Category  logprob  loglift
2          good  348.000000  348.000000  Default  30.0000  30.0000
7    restaurant  220.000000  220.000000  Default  29.0000  29.0000
164        meat   73.000000   73.000000  Default  28.0000  28.0000
171      salads   68.000000   68.000000  Default  27.0000  27.0000
266      prices   98.000000   98.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
2          good    1.916550  348.744080  Topic10  -5.4292  -1.5928
255     variety    1.623991   57.375446  Topic10  -5.5949   0.0463
24        lunch    1.524645   58.362350  Topic10  -5.6580  -0.0339
57       people    1.448304   41.592709  Topic10  -5.7094   0.2535
4         place    1.479556  248.475739  Topic10  -5.6880  -1.5126

[795 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
953       8  0.628745         aaaa
2089      6  0.806673     abundant
591       5  0.603909  accommodate
591       7  0.301954  accommodate
1767      2  0.209024  accompanied
...     ...       ...          ...
667       1  0.170520        young
667       2  0.170520        young
667       4  0.170520        young
667       8  0.341040        young
1220      1  0.731181        yummy

[1763 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 7, 1, 10, 8, 6, 2, 4, 9])